# 02_ocr_testing.ipynb
## Pruebas de Extracción OCR

Objetivo: Prototipar y validar OCR (Landing AI vs DeepSeek)

## 1. Configuración de entorno (.env)

Cargamos las variables desde el archivo `.env` en la raíz del proyecto (por ejemplo, `LANDING_AI_API_KEY`), para no tener que pegar la API key directamente en el notebook.

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

# --- Cargar variables de entorno desde .env ---
# Buscamos el .env en la raíz del proyecto (una carpeta por encima de 'notebooks')
PROJECT_ROOT = Path.cwd().parent
env_path = PROJECT_ROOT / ".env"

if env_path.exists():
    load_dotenv(env_path)
    print(f"ℹ️ Cargando variables de entorno desde: {env_path}")
else:
    # Fallback: dejar que python-dotenv busque un .env en el árbol de directorios
    load_dotenv()
    print("ℹ️ Archivo .env no encontrado en la raíz del proyecto; intentando carga por defecto.")

print("✅ Variables de entorno cargadas (si estaban definidas).")


ℹ️ Cargando variables de entorno desde: c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent\.env
✅ Variables de entorno cargadas (si estaban definidas).


In [2]:
import sys
import logging
from pathlib import Path
import json

# --- Configuración de Path ---
# Añadir el directorio PADRE (la raíz del proyecto) al path
# Esto es crucial porque el notebook está en 'notebooks/' y 'src' está en el nivel superior.
# .../Agente-Multimodal-Liquidador/
#    ├── notebooks/ (Aquí está el notebook)
#    └── src/       (Esto es lo que queremos importar)
#
# Path.cwd() -> .../notebooks
# Path.cwd().parent -> .../ (la raíz del proyecto)
#
# Así, Python puede encontrar 'src'
PROJECT_ROOT = str(Path.cwd().parent)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# --- Importaciones del Proyecto ---
# Ahora que el __init__.py de 'extractors' está corregido, podemos importar todo.


# Importa utilidades
from src.utils.logger import get_logger
from src.utils.config import (
    EXCEL_IMAGES_DIR, 
    PDF_IMAGES_DIR, 
    EXTRACTED_TEXT_DIR,     # Lo usaremos para probar el parser
    EXTRACTED_TABLES_DIR    # Para ver la salida del parser
)

# --- Configuración de Logging ---
# Configura el logger para que puedas ver los mensajes INFO en la salida del notebook
logger = get_logger(__name__)
logging.basicConfig(level=logging.INFO) 

print("✅ Dependencias del proyecto importadas correctamente.")
print(f"Raíz del proyecto establecida en: {PROJECT_ROOT}")

✅ Dependencias del proyecto importadas correctamente.
Raíz del proyecto establecida en: c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent


## 2. Verificar Imágenes Disponibles

In [3]:
# Listar imágenes disponibles
excel_images = list(EXCEL_IMAGES_DIR.glob("*.png"))
pdf_images = list(PDF_IMAGES_DIR.glob("*.png"))
all_images = excel_images + pdf_images

print(f"Imágenes de Excel: {len(excel_images)}")
print(f"Imágenes de PDF: {len(pdf_images)}")
print(f"Total: {len(all_images)}")

if all_images:
    print(f"\nPrimeras imágenes:")
    for img in all_images[:5]:
        print(f"  - {img.name}")
else:
    print("\n⚠️ No hay imágenes. Ejecuta notebook 01 primero.")

Imágenes de Excel: 8
Imágenes de PDF: 1
Total: 9

Primeras imágenes:
  - 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r0_c0.png
  - 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r0_c200.png
  - 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r0_c250.png
  - 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r100_c0.png
  - 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r150_c0.png


## 3. Inicializar OCR Extractor

In [5]:
# Crear extractor
# Cambiar provider en .env: OCR_PROVIDER=landing_ai o deepseeker
from src.extractors import OCRExtractor

extractor_landing = OCRExtractor(provider="landing_ai")
print("✅ Landing AI extractor inicializado")


ImportError: cannot import name 'LANDING_AI_API_KEY' from 'src.utils.config' (c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent\src\utils\config.py)

## 4. Prueba OCR: Landing AI

In [ ]:
import requests
import sys
import os
from pathlib import Path

# --- (Opcional) Cargar variables de tu notebook si están disponibles ---
# Esto intenta encontrar la variable 'INPUT_PDF_DIR' de tu notebook.
try:
    # Esta variable se define en la primera celda de 02_ocr_testing.ipynb
    # Si no está definida, creará una ruta relativa.
    if 'INPUT_PDF_DIR' not in locals():
        PROJECT_ROOT = Path.cwd().parent
        INPUT_PDF_DIR = PROJECT_ROOT / "data" / "input" / "pdf"
        print("Variable 'INPUT_PDF_DIR' no encontrada, usando ruta por defecto.")
        
except NameError:
    # Fallback si 'Path' no está importado
    INPUT_PDF_DIR = Path.cwd().parent / "data" / "input" / "pdf"
    print("Variable 'INPUT_PDF_DIR' no encontrada, usando ruta por defecto.")




MI_API_KEY = os.getenv("LANDING_AI_API_KEY", "pega_tu_api_key_aqui")

# =================================================================


# --- No edites debajo de esta línea ---

def probar_conexion_con_documento():
    """
    Prueba la conexión contra el endpoint 'Agentic Document Extraction'
    enviando un documento PDF real.
    """
    
    # 2. Verificar si la clave fue reemplazada
    if MI_API_KEY == "pega_tu_api_key_aqui" or MI_API_KEY == "":
        print("\n-----------------------------------------------------")
        print("❌ ERROR: Edita este bloque de código primero.")
        print("Pega tu API key real en la variable 'MI_API_KEY'.")
        print("-----------------------------------------------------")
        return # Salir de la función

    # 3. Definir el documento a enviar
    # (Este es el PDF original, no el PNG)
    pdf_file_name = "25105 25106 25107 & 25108 MV SKY KNIGHT - VAL TISUR.pdf"
    documento_path = INPUT_PDF_DIR / pdf_file_name

    if not documento_path.exists():
        print(f"\n❌ ERROR: No se encontró el archivo PDF en:")
        print(f"  {documento_path}")
        print("   Asegúrate de que el archivo exista en 'data/input/pdf/'")
        return
    
    # 4. Endpoint y Headers
    URL_DEL_SERVICIO = "https://api.va.landing.ai/v1/ade/parse"
    headers = {
        "Authorization": f"Bearer {MI_API_KEY}"
    }

    # 5. Preparar los archivos para el POST
    # (La API espera 'multipart/form-data')
    files_payload = {
        'document': (documento_path.name, open(documento_path, 'rb'), 'application/pdf'),
        'model': (None, 'dpt-2-latest'),
    }

    print(f"Intentando conectar a: {URL_DEL_SERVICIO} ...")
    print(f"Enviando documento: {documento_path.name}")

    try:
        # 6. Hacemos la llamada POST con el archivo
        response = requests.post(URL_DEL_SERVICIO, headers=headers, files=files_payload)

        # 7. Interpretar la respuesta
        
        if response.status_code == 401:
            print("\n-----------------------------------------------------")
            print("❌ CONEXIÓN FALLIDA: 401 Unauthorized")
            print("La API key que pegaste en el script es INCORRECTA,")
            print("ha expirado o no pertenece a este servicio.")
            print("-----------------------------------------------------")
            
        elif response.status_code == 200:
            print("\n-----------------------------------------------------")
            print(f"✅ ¡CONEXIÓN EXITOSA! (Recibido: 200 OK)")
            print("El servidor aceptó tu API key y procesó el documento.")
            print("\nRespuesta (primeros 500 caracteres del Markdown):")
            print("--------------------------------------------------")
            try:
                # Extraer el 'markdown' del JSON de respuesta
                extracted_text = response.json().get('markdown', 'No se encontró texto.')
                print(extracted_text[:500] + "...")
            except requests.exceptions.JSONDecodeError:
                print("Error: No se pudo decodificar la respuesta JSON del servidor.")
            print("--------------------------------------------------")
            
        elif response.status_code == 422:
             print("\n-----------------------------------------------------")
             print(f"❌ ERROR DE PROCESAMIENTO: 422 Unprocessable Entity")
             print("Tu API key es VÁLIDA, pero el servidor no pudo procesar")
             print(f"el archivo '{documento_path.name}'.")
             print(f"Respuesta del servidor: {response.json()}")
             print("-----------------------------------------------------")

        else:
            print(f"\nRespuesta inesperada del servidor: {response.status_code}")
            print(response.text)

    except requests.exceptions.RequestException as e:
        print(f"\nError de conexión de red (¿Estás conectado a internet?): {e}")
    finally:
        # Asegurarse de cerrar el archivo
        if 'files_payload' in locals():
            files_payload['document'][1].close()

# --- Ejecutar la prueba ---
probar_conexion_con_documento()

In [ ]:
def extract_ade_from_pdf(pdf_path: Path) -> dict:
    """
    Llama al servicio ADE de Landing AI con un archivo PDF.
    Devuelve un diccionario con el estado y el texto markdown.
    """
    URL_DEL_SERVICIO = "https://api.va.landing.ai/v1/ade/parse"
    headers = {"Authorization": f"Bearer {MI_API_KEY}"}

    if not pdf_path.exists():
        logger.error(f"Archivo PDF no encontrado: {pdf_path}")
        return {"status": "error", "message": "Archivo no encontrado"}

    # 'rb' = read binary
    with open(pdf_path, 'rb') as f_pdf:
        files_payload = {
            'document': (pdf_path.name, f_pdf, 'application/pdf'),
            'model': (None, 'dpt-2-latest'),
        }
        
        try:
            response = requests.post(URL_DEL_SERVICIO, headers=headers, files=files_payload, timeout=60)
            
            if response.status_code == 200:
                logger.info(f"ADE procesó exitosamente: {pdf_path.name}")
                return {
                    "status": "success",
                    "provider": "landing_ai_ade",
                    "text": response.json().get('markdown', ''), # Extraemos el markdown
                    "raw_response": response.json()
                }
            else:
                logger.warning(f"Error de API {response.status_code} para {pdf_path.name}: {response.text}")
                return {"status": "error", "message": f"Error API {response.status_code}: {response.text}"}

        except requests.exceptions.RequestException as e:
            logger.error(f"Error de red al contactar ADE: {e}")
            return {"status": "error", "message": str(e)}

print("✅ Función extract_ade_from_pdf definida.")
if MI_API_KEY == "09876543": # Revisa si sigue la clave de ejemplo
    print("⚠️ ADVERTENCIA: Estás usando la API key de ejemplo '09876543'.")
    print("   Reemplázala con tu clave real para que funcione.")
elif len(MI_API_KEY) < 10: # Revisa si la clave es muy corta
    print(f"⚠️ ADVERTENCIA: La API key '{MI_API_KEY}' parece incorrecta o incompleta.")

In [ ]:
# Celda 7 (Reemplazada): Prueba de extracción con ADE

# CORRECCIÓN: Buscamos un PDF para probar
pdf_files = list(INPUT_PDF_DIR.glob("*.pdf"))

if pdf_files:
    test_pdf_path = pdf_files[0] # Usamos el primer PDF disponible
    
    print(f"Probando extracción ADE con: {test_pdf_path.name}")
    
    # Llamar a nuestra nueva función
    result_ade = extract_ade_from_pdf(test_pdf_path)
    
    print(f"\nResultado:")
    print(f"  Status: {result_ade.get('status')}")
    print(f"  Provider: {result_ade.get('provider')}")
    
    if result_ade.get('status') == 'success':
        text = result_ade.get('text', '')
        print(f"\n  Markdown extraído (primeros 500 chars):")
        print("="*50)
        print(f"{text[:500]}...")
        print("="*50)
        print(f"  Total chars: {len(text)}")
    else:
        print(f"  ⚠️ Error: {result_ade.get('message')}")
else:
    print("⚠️ No hay archivos PDF en data/input/pdf/. Añade uno para probar.")

## 6. Extracción de Estructura

In [ ]:
# Celda 18 (Reemplazada): Procesar TODOS los PDFs con ADE

print("Procesando todos los PDFs con ADE...")
print("="*60)

# CORRECCIÓN: Buscar PDFs en data/input/pdf
all_pdfs = list(INPUT_PDF_DIR.glob("*.pdf"))

results_summary = {
    'success': 0,
    'error': 0,
    'total': len(all_pdfs)
}

all_ade_results = [] # Para guardar los resultados

for idx, pdf_file in enumerate(all_pdfs, 1):
    print(f"\n[{idx}/{len(all_pdfs)}] {pdf_file.name}")
    
    result = extract_ade_from_pdf(pdf_file)
    all_ade_results.append(result) # Guardar el resultado
    
    if result.get('status') == 'success':
        text = result.get('text', '')
        print(f"  ✅ Extraído: {len(text)} caracteres de markdown")
        results_summary['success'] += 1
    else:
        print(f"  ❌ Error: {result.get('message')}")
        results_summary['error'] += 1

print(f"\n{'='*60}")
print(f"RESUMEN:")
print(f"  Exitosos: {results_summary['success']}")
print(f"  Errores: {results_summary['error']}")
print(f"  Total: {results_summary['total']}")

## 7. Parser: Estructuración JSON

In [ ]:
# Celda 19 (Reemplazada): Guardar resultados ADE

# CORRECCIÓN: Guardar los resultados de 'all_ade_results'
if 'all_ade_results' in locals() and all_ade_results:
    output_file = Path('ade_test_output.json')
    
    # Guardamos la lista completa de resultados
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_ade_results, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Resultados guardados en: {output_file}")
    
    # Mostrar un resumen del primer resultado (si existe)
    if all_ade_results[0]['status'] == 'success':
        print(f"\nPrimer resultado guardado (primeros 500 chars):")
        print(json.dumps(all_ade_results[0]['text'], ensure_ascii=False, indent=2)[:500] + "...")
    else:
        print(f"\nPrimer resultado fue un error:")
        print(json.dumps(all_ade_results[0], ensure_ascii=False, indent=2))
else:
    print("⚠️ No hay 'all_ade_results' para guardar. Ejecuta la celda anterior.")

## 8. Comparación: Landing AI vs DeepSeek

## 9. Procesar todas las imágenes

In [ ]:
import requests
from pathlib import Path
# Asumimos que 'MI_API_KEY' y 'logger' ya están definidos en celdas anteriores.

def extract_ade_from_image(image_path: Path) -> dict:
    """
    Llama al servicio ADE de Landing AI con un archivo de IMAGEN (PNG).
    Devuelve un diccionario con el estado y el texto markdown.
    """
    URL_DEL_SERVICIO = "https://api.va.landing.ai/v1/ade/parse"
    headers = {"Authorization": f"Bearer {MI_API_KEY}"}

    if not image_path.exists():
        logger.error(f"Archivo de imagen no encontrado: {image_path}")
        return {"status": "error", "message": "Archivo no encontrado"}

    # 'rb' = read binary
    with open(image_path, 'rb') as f_img:
        
        # --- ¡MODIFICACIÓN CLAVE! ---
        # Cambiamos el MIME type a 'image/png'
        files_payload = {
            'document': (image_path.name, f_img, 'image/png'), 
            'model': (None, 'dpt-2-latest'),
        }
        # --- Fin de la Modificación ---
        
        try:
            # Enviamos la imagen al mismo endpoint
            response = requests.post(URL_DEL_SERVICIO, headers=headers, files=files_payload, timeout=60)
            
            if response.status_code == 200:
                logger.info(f"ADE procesó exitosamente la imagen: {image_path.name}")
                return {
                    "status": "success",
                    "provider": "landing_ai_ade",
                    "text": response.json().get('markdown', ''), # Extraemos el markdown
                    "raw_response": response.json()
                }
            else:
                logger.warning(f"Error de API {response.status_code} para {image_path.name}: {response.text}")
                return {"status": "error", "message": f"Error API {response.status_code}: {response.text}"}

        except requests.exceptions.RequestException as e:
            logger.error(f"Error de red al contactar ADE con imagen: {e}")
            return {"status": "error", "message": str(e)}

print("✅ Función extract_ade_from_image definida.")
print("   Lista para probar con los archivos PNG.")

In [ ]:
# Celda de prueba: Procesar TODAS las imágenes PNG con ADE

# (La variable 'all_images' ya fue definida en la celda 
#  "Listar imágenes disponibles" que pegaste en tu prompt)

print(f"\nProbando extracción ADE con las {len(all_images)} imágenes PNG encontradas...")
print("="*60)

results_summary = {
    'success': 0,
    'error': 0,
    'total': len(all_images)
}
all_ade_image_results = [] # Para guardar los resultados

if not all_images:
    print("⚠️ No se encontraron imágenes en 'all_images'.")
    print("   Asegúrate de ejecutar la celda 'Listar imágenes disponibles' primero.")
else:
    for idx, img_path in enumerate(all_images, 1):
        print(f"\n[{idx}/{len(all_images)}] Procesando: {img_path.name}")
        
        # Llamar a nuestra NUEVA función de imágenes
        result = extract_ade_from_image(img_path)
        all_ade_image_results.append(result) # Guardar el resultado
        
        if result.get('status') == 'success':
            text = result.get('text', '')
            print(f"  ✅ Extraído: {len(text)} caracteres de markdown")
            
            # Opcional: Imprimir un fragmento del texto
            print(f"  Fragmento: {text.replace(chr(10), ' ')[10:70]}...")
            
            results_summary['success'] += 1
        else:
            print(f"  ❌ Error: {result.get('message')}")
            results_summary['error'] += 1

    print(f"\n{'='*60}")
    print(f"RESUMEN DE PRUEBA DE IMÁGENES:")
    print(f"  Exitosos: {results_summary['success']}")
    print(f"  Errores: {results_summary['error']}")
    print(f"  Total: {results_summary['total']}")

In [ ]:
import json
from pathlib import Path

print("\nGuardando los resultados de ADE en un archivo JSON...")

# Comprueba si la variable 'all_ade_image_results' existe y tiene contenido
if 'all_ade_image_results' in locals() and all_ade_image_results:
    
    # Definimos el nombre del archivo de salida
    output_file = Path('ade_image_test_output.json') 
    
    try:
        # Guardamos la lista completa de resultados
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(all_ade_image_results, f, ensure_ascii=False, indent=2)
        
        print(f"✅ ¡Éxito! {len(all_ade_image_results)} resultados guardados en:")
        # .resolve() muestra la ruta completa donde se guardó el archivo
        print(f"   {output_file.resolve()}") 
    
    except Exception as e:
        print(f"❌ Error al guardar el archivo JSON: {e}")

else:
    print("⚠️ No se encontró la variable 'all_ade_image_results' o está vacía.")
    print("   (Esto no debería pasar si la celda anterior se ejecutó bien).")